In [1]:
import pandas as pd
import requests

In [4]:
# Define parameters
start_time = "2000-01-01"
end_time = "2025-01-01"
min_magnitude = 4.5
bbox = [26.5, 88.5, 28.5, 92.0]  # approx Bhutan bounding box [min_lat, min_lon, max_lat, max_lon]

url = f"https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "geojson",
    "starttime": start_time,
    "endtime": end_time,
    "minlatitude": bbox[0],
    "minlongitude": bbox[1],
    "maxlatitude": bbox[2],
    "maxlongitude": bbox[3],
    "minmagnitude": min_magnitude,
}

response = requests.get(url, params=params)
data = response.json()

# Parse earthquakes
eq_list = []
for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    eq_list.append({
        "time": pd.to_datetime(props["time"], unit="ms"),
        "latitude": coords[1],
        "longitude": coords[0],
        "depth_km": coords[2],
        "magnitude": props["mag"],
        "place": props["place"]
    })

earthquakes_df = pd.DataFrame(eq_list)

In [5]:
earthquakes_df

,time,latitude,longitude,depth_km,magnitude,place
0,2021-11-07 16:20:46.399,27.2710,88.7826,10.00,4.6,"17 km ESE of Gangtok, India"
1,2021-04-05 15:19:57.976,27.1863,88.9412,10.00,5.2,"35 km NNW of Samtse, Bhutan"
2,2019-10-07 12:35:46.401,27.2930,91.3145,10.00,4.7,"7 km ENE of Mongar, Bhutan"
3,2017-03-26 21:42:11.940,27.1419,88.5534,24.39,4.5,"4 km SSE of Rangpo, India"
4,2015-07-01 02:42:40.790,27.2728,91.2869,35.46,4.5,"4 km E of Mongar, Bhutan"
5,2015-06-28 01:05:28.560,26.6384,90.4100,26.00,5.1,"18 km N of B?sugaon, India"
6,2013-12-04 13:05:19.880,26.5885,89.5628,13.60,4.7,"12 km NNE of Al?pur Du?r, India"
7,2012-09-18 12:26:25.280,27.3300,88.6420,42.20,4.6,"2 km E of Gangtok, India"
8,2012-07-10 13:03:55.050,27.0740,90.6630,35.00,4.7,"15 km S of Shemgang, Bhutan"
9,2012-03-01 17:25:48.650,27.4470,91.2370,53.60,4.6,"19 km N of Mongar, Bhutan"


In [17]:
from geopy.distance import geodesic

def is_within_radius(eq_lat, eq_lon, cell_lat, cell_lon, km=50):
    return geodesic((eq_lat, eq_lon), (cell_lat, cell_lon)).km <= km


In [ ]:
model_df["recent_quake_within_50km"] = 0

for idx, row in model_df.iterrows():
    date = row["date"]
    lat = row["latitude"]
    lon = row["longitude"]

    recent_eqs = earthquakes_df[
        (earthquakes_df["time"] >= date - pd.Timedelta(days=30)) &
        (earthquakes_df["time"] <= date)
    ]

    for _, eq in recent_eqs.iterrows():
        if is_within_radius(eq["latitude"], eq["longitude"], lat, lon, km=50):
            model_df.at[idx, "recent_quake_within_50km"] = 1
            break


In [7]:
features = ["temperature", "precipitation", "wind_u", "wind_v",
            "recent_quake_within_50km", "max_magnitude_last_30days"]


In [16]:
realtime_eq_df

""
